In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [4]:
# already sorted by session, timestamp
df = pd.read_parquet("input/transactions.parquet")
df.info()

KeyboardInterrupt: 

In [ ]:
_next = 0
vocab: Dict[str, int] = {
    "<s>": 0,
    "<pad>": 1,
    "</s>": 2,
    "<unk>": 3,
    "<mask>": 4,
    "<click>": 5
    "<cart>": 6,
    "<order>": 7,
}


In [ ]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/sequences.parquet", index=False)